# Fundamentals

## Joint Assembly

<figure style="text-align: center;">
    <img src="./Resources/Axis3.png" style="width:50%; margin: auto;">
    <figcaption align="center"> <b>Figure 3.1.: </b>Axis assembly of Joint-3 in LARA8. Adapted from [] </figcaption>
</figure>

The general schema of the robot's axis is presented in Figure 3.1; a DC motor drives the axis and is the source of torque. The Harmonic Drive is the gearbox that significantly steps down the rotational speed while stepping up the torque. There are optical encoders on both sides of the Harmonic Drive; these encoders will henceforth be referred to as *"motor-side"* and *"load-side"* encoders. The link is connected to the gearbox's output shaft via a rigid coupling. Currently, In the robot software, the motor side encoder is used only to measure the joint velocity and the load side encoder is used to measure the joint angle. However, this thesis aims to find another application for these encoders.

### DC Motor

The motor converts electric current  to mechanical torque which drives the robot's joint. According to **Taghirad et al.**, The relationship between the input current and the output torque of a motor can be expressed as
\begin{equation}
K_m \times I = J_m \ddot\theta + T_{f_m} + T_out,
\label{eq:3.1} \tag{3.1}
\end{equation}
where  $I$ is the motor current, and $K_m$  is the motor-torque-constant, $J$ is the motor inertia, $T_out$ is the output torque, $\ddot\theta$ is the acceleration of the motor shaft, and $T_{f_m}$ is the motor's frictional torque. 

### Encoders
The motor-side and load-side encoders are both 24 bit optical encoders. These encoders provide both absolute and incremental measurements. The absolute track is used only during startup and afterwards the encoder uses the incremental measurements. These encoders are used in the feedback loop of the control system for measuring position and velocity of the joint. If the gearbox is rigid, the readings from both the encoders will be
\begin{equation}
\frac{\theta_m}{N} = \theta_l,
\label{eq:3.2} \tag{3.2}
\end{equation} 
where $N$ is the gear ratio, $\theta_m$ is the motor-side encoder reading and $\theta_l$ is the load-side encoder reading. This implies that in scenarios where the gearbox is not prone to deformation, only one encoder is required.

### Harmonic Drive Transmission

<figure style="text-align: center;">
<img src="./Resources/harmonic.jpg" style="width:30%">
<figcaption align = "center"> <b>Figure 3.2.: </b>Elements of Harmonic Drive Transmission </figcaption>
</figure>

The Harmonic Drive Transmission (HDT) comprises three main parts: a wave generator, flexible spline or flex spline and circular spline. The wave generator is the input of the harmonic drive transmission and has an elliptical shape; when rotated, causes the flexible spline to follow its elliptical shape and to engage with the circular spline; although flexible, the flex spline is torsionally stiff enough to transmit large torques. According to **Taghirad et al.**, HDT's advantages include "high torque capacity, concentric geometry, lightweight and compact design, zero backlash, high efficiency, and back driveability"[]. But they suffer from "high flexibility, resonance vibration, friction and structural damping nonlinearities"[]. 

#### Stiffness

When transmitting torque, the flexspline being flexible undergoes torsional deformation. A plot of the torsion angle against the torque is showin in Figure 3.3(a). An approximation of this torque-torsional relationship of the HDT is usually provided by the manaufacturer in the datasheet in the form of a piecewise linear function as shown in Figure 3.3(b).

<figure style="text-align: center;">
    <div style="display: inline-block;">
        <img src="./Resources/torque-torsion-angle-diagram.png" style="width:100%;">
        <figcaption>(a) Torsion vs Torque </figcaption>
    </div>
    <div style="display: inline-block; margin-left: 20px;">
        <img src="./Resources/spring-constant-diagram.png" style="width:100%;">
        <figcaption>(b) Piecewise linear approximation of Torsion vs Torque</figcaption>
    </div>
    <figcaption style="margin-top: 10px;"><b>Figure 3.3.: </b>Plots from datasheet of SHG-SHF_GearUnits []</figcaption>
</figure>

HDT's do not follow Equation 3.1 due to this behaviour and hence, the two seprate encoders are generally used in such systems involving HDT's. The relation between motor-side and load-side encoder readings in such systems is given by 
\begin{equation}
\frac{\theta_m}{N } = \theta_l - \theta_e,
\label{eq:3.3} \tag{3.3}
\end{equation}
where $ \theta_e$ is the torsion angle. Below is an algorithmic implementation of this torsion-torque relationship.

In [3]:
def torsion_to_torque(encoder_error, T1, T2, T3, K1, K2, K3):
    # encoder error in counts
    # torque in Nm
    count_to_radian = 2 * np.pi / (2**24)
    torsion = torsion * count_to_radian
    THETA1 = T1 / K1
    THETA2 = THETA1 + (T2 - T1) / K2
    return (torsion * K1 if torsion < THETA1 else
            T1 + (torsion - THETA1) * K2 if torsion < THETA2 else
            T2 + (torsion - THETA2) * K3)

##  LARA 8 Kinematic Description


![convert to latex](Resources/Lara8.png "Lara8")


The robot used for the experiments in this thesis is LARA 8, a lightweight collaborative robotic
arm from the company NEURA Robotics GmbH. It is a 6-axis serial meanipulator
that has six independently actuated revolute joints and seven links. The maximum payload is 8kg. Figure[] shows the DH
frames, the DH parameters and the actuated joints of LARA 8. The DH-parameters
are given in Table [cite]

| frame | theta (rad) | d (m) | a (m) | alpha (rad) |
|---|---|---|---|---|
| K1 | 0 | 0.26 | 0 | $\frac{\pi}{2}$ |
| K2 | $\frac{\pi}{2}$ | 0 | 0.6 | $\pi$ |
| K3 | $\frac{\pi}{2}$ | 0 | 0 | $\frac{\pi}{2}$ |
| K4 | 0 | 0.7 | 0 | $-\frac{\pi}{2}$ |
| K5 | 0 | 0 | 0 | $\frac{\pi}{2}$ |
| K6 | 0 | 0.1735 | 0 | 0 |


## Kinematics

The kinematics of a robot used in the thesis are introduced in this section along with some basic concepts and algorithmic implementations. The first subsection discusses the conversions between coordinate frames and the selected convention. Next, a brief explanation of the Geometric Jacobian computation procedure and forward kinematics is given.

### Denavit-Hartenberg Convention and Transformation Matrix

Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the 
kinematic geometry of robotic arms is the Denavit-Hartenberg (DH) convention <cite id="gx4bn">(Ghosal, 2006)</cite><cite id="10z4u">(Denavit & Hartenberg, 2021)</cite> and is followed in this thesis.

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)
\end{equation}

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5">(Ghosal, 2006)</cite>.

In [1]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

### Forward Kinematics

In a robotic manipulator, the links are generally rigid bodies that form the robot’s structure, while the joints are the movable components that join two or multiple links. Joints can be  with an actuator (active joints), or without an actuator (passive joints).Joints commonly found in serial robots are usually prismatic or revolute. Each type of such joint adds one degree of freedom (dof) to the system: the joint position $d_i$ for prismatic joints or the joint angle $q_i$ for revolute joints. The entire kinematic structure is typically referred to as a chain, which can be of two types:

* Open chain: In this configuration, the two extremes, also known as the base and the end-effector are connected by a single path in the structure.
* Closed chain: The kinematic structure of this configuration has loops.

The robot used in this thesis consists only of active revolute joint in an open chain configuration.

A common problem in robotics is determining the Cartesian position and orientation $\mathbf{x} \in \mathbb{R}^6$ of a specific point on a link given the joint angles $\mathbf{q} \in \mathbb{R}^n$. This problem is known as forward kinematics, which can be efficiently solved using homogeneous transformations. The position of the origin of the end-effector’s reference frame $\text{RF}_e$ in the base frame $\text{RF}_b$ can be determined using the following equation:

$$
{}^b\mathbf{T}_e(\mathbf{q}) = {}^b\mathbf{T}_0 \, {}^0\mathbf{T}_1(q_1) \cdots {}^{n-1}\mathbf{T}_e(q_n) \, {}^n\mathbf{T}_e,
$$

where the two constant homogeneous transformations required to convert a point from frame 0 to the base frame and from the end-effector frame to frame $n$, respectively, are denoted by ${}^b\mathbf{T}_0$ and ${}^n\mathbf{T}_e$.

 The following algorithmic implementation computes the transformation between all reference frames in the robot defined in the dh parameters.

In [3]:
def compute_forward_kinematics(joint_angles, dh_params):
    """
    Compute the forward kinematics for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: List of transformation matrices from base to each joint and the final transformation matrix T_0_n
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    T_0_n = transformations[-1]  # Final transformation matrix to the TCP (end-effector)
    return transformations, T_0_n

###  Geometric Jacobian

The geometric Jacobian matrix, $J(q) ∈ R^(6×n)$, transforms the joint space velocities $q̇$, into Cartesian space velocities $ẋ$. This relationship is represented by the equation:
\begin{equation}
\dot{\mathbf{x}} = 
\begin{bmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{bmatrix} 
= \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}},
\end{equation}
where $\dot p = [ \dot x, \dot y, \dot z ]^T $represents the linear velocities and $\omega = [ \omega_x, \omega_y, \omega_z ]^T$ represents the angular velocities of the robot's end-effector in the base frame. The geometric Jacobian is calculated based on the robot's geometry at a specific instance. An elaborate theortical background can be found in [], an algorithmic implementation is shown below. 

In [4]:
def compute_geometric_jacobian(transformations):
    """
    Compute the geometric Jacobian for a robot given the transformation matrices.

    :param transformations: List of transformation matrices from the base to each joint
    :return: Geometric Jacobian matrix (6 x N) in the base frame
    """
    num_joints = len(transformations) - 1  # Excluding the base frame
    jacobian_base = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian in the base frame
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian_base[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian_base[3:, i] = z_i  # Angular part

    return jacobian_base

In [5]:
def convert_jacobian_to_tcp(jacobian_base, T_0_n):
    """
    Convert a Jacobian from the base frame to the TCP frame.
    
    :param jacobian_base: Jacobian matrix in the base frame (6 x 6)
    :param T_0_n: Transformation matrix from the base to the TCP (4 x 4)
    :return: Jacobian matrix (6 x 6) in the TCP frame
    """
    R_0_n = T_0_n[:3, :3]
    T_base_to_tcp = np.block([
        [R_0_n.T, np.zeros((3, 3))],
        [np.zeros((3, 3)), R_0_n.T]
    ])
    
    jacobian_tcp = np.dot(T_base_to_tcp, jacobian_base)
    return jacobian_tcp

##  Dynamics

This section lays out the theoretical groundwork for understanding the dynamics of serial manipulators. First, equation of motion for a generic open-chain robotic manipulator is presented. These differential equations form the dynamic model of the robot, describing the relationship between joint positions ($q$), velocities ($\dot{q}$), and accelerations ($\ddot{q}$) with the torques ($\tau$) applied to these joints by actuators.

Following this, the topic of friction, its modeling and algorithmic implementation is introduced. The final part of the section addresses the calculation and algorithmic implementation necessary for converting torques at the Tool Center Point (TCP) to joint torques and vice versa.

###  Equations of motion

The general equation of motion for the robot under study, which is an open chain manipulator is given below. This equation presents the joint torque $\tau$ required to move the robot's joints in accordance with a desired trajectory with joint positions $q$,joint velocities $\dot{q}$, and joint accelerations $\ddot{q}$. 
\begin{equation}
 \underbrace{M(\mathbf{q}) \ddot{\mathbf{q}} + \mathbf{C}(\mathbf{q}, \dot{\mathbf{q}}) + \mathbf{g}(\mathbf{q}) + \mathbf{\tau_f}}_{\text{Torque Feedforward}}+ \mathbf{\tau_{ext}}= \mathbf{\tau}.
\label{eq:3.8} \tag{3.8}
\end{equation}
Here, $M (q) ∈ R^{n×n}$ is a symmetric and positive definite matrix representing the robot’s inertia, $C(q, q̇) ∈ R^n$ is the vector containing the Coriolis, centrifugal and gyroscopic terms and $g(q) ∈ R^n$ is the gravity vector, $\mathbf{\tau_f}$ is the frictional torque and $\mathbf{\tau_{ext}}$ is the torque in the joint due to interaction of the robot with the environment. The term labeled as 'Torque Feedforward' in the equation can be computed for the desired trajectory, providing a preliminary estimate of the joint torques. This feedforward torque is used by the PID controllers in each joint to achieve the desired joint position, velocity, and acceleration. Although the controller will adjust the torque applied to the joint as needed, the feedforward torque serves as an initial estimate. The detailed derivation and more discussions can be found in [B. Siciliano, L.Sciavicco, L. Villani, and G. Oriolo. Robotics: Modelling, planning
and control. Springer, 2009.]


###  Friction Model

Friction occurs in most mechanical systems where there is rleative motion between two mechanical components. Inorder to provide a good torque estimate following equation 3.8, it is essential to have a good model of the joitn's friction. However friction is very complex to model accurately and is affected by many factors such as joint velocity, temperature, load, etc. Inorder to model the frictional torque, many simplifications have to be done. According to [https://www.sciencedirect.com/science/book/9780128136836] friction dynamics can be simplified into the following four components.

#### Static Friction

Static friction is the component of friction which is present between two surfaces in contact, when there is no relative motion and an external force is applied. Static friction acts in the direction opposite to the applied force that is attempting to initiate motion. It's magnitude is equal to the external force ($f_e$) until the external force exceeds the maximum static frictional force ($f_s$), beyond which static friction is overcome, and motion begins.
\begin{equation}
[
T_f =
\begin{cases} 
f_e, & |f_e| < f_s \\
f_s \delta(\dot{x}) \text{sgn}(f_e), & |f_e| \geq f_s 
\end{cases}
]
\label{eq:3.9} \tag{3.9}
\end{equation}
$\text{where}$
\begin{equation}
[
\delta(\dot{x}) = \begin{cases} 
1, & \dot{x} = 0 \\
0, & \dot{x} \neq 0 
\end{cases}
\quad , \quad
\text{sgn}(f_e) = \begin{cases} 
+1, & f_e > 0 \\
-1, & f_e < 0 
\end{cases}
]
\label{eq:3.10} \tag{3.10}
\end{equation}

#### Coulomb friction
Coulomb friction opposes the motion and is proportional to the normal contact force ($f_n$). 

\begin{equation}
T_f =  \mu f_n \text{sgn}(\dot{x}),
\label{eq:3.11} \tag{3.11}
\end{equation}
where $\mu$ is a constant called friction coefficient.

#### Viscous friction
This is a velocity dependent component of friction and is proportional to velocity. It is given by the equation,

\begin{equation}
T_f = f_v \dot(x),
\label{eq:3.12} \tag{3.12}
\end{equation}
where $f_v$ is a constant called viscous coefficient.

#### Friction model

Multiple friction models have been studied [] which cover all these components, however the friction model used in this thesis is from [] and is given by 

\begin{equation}
f(v) = a_1 (\tanh(b_1 v) - \tanh(b_2 v)) + a_2 \tanh(b_3 v) + a_3 v,
\label{eq:3.13} \tag{3.13}
\end{equation}

where $a_1,a_2,a_3,b_1,b_2,b_3$ are all positive paramters which are found emperically.

###  Conversion of TCP forces to Joint torques


The conversion of external forces and torques applied to the end-effector of a 6-DOF robot to joint torques is achieved through the Jacobian matrix $\mathbf{J}(\mathbf{q})$, which relates joint velocities to the end-effector linear and angular velocities.

#### External Force and Torque Representation

External forces $\mathbf{F}_{\text{ext}}$ and torques $\mathbf{\tau}_{\text{ext}}$ are combined into a wrench vector $\mathbf{W}_{\text{ext}}$:

$$
\mathbf{W}_{\text{ext}} = \begin{bmatrix} \mathbf{F}_{\text{ext}} \\ \mathbf{\tau}_{\text{ext}} \end{bmatrix}
$$


#### Conversion to Joint Torques

The joint torques $\mathbf{\tau}_{\text{joint}}$ are computed by:

$$
\mathbf{\tau}_{\text{joint}} = \mathbf{J}^\top(\mathbf{q}) \mathbf{W}_{\text{ext}}
$$

This equation projects the external wrench into the joint space, providing the necessary torques required at each joint to counteract the external forces and torques.


Note that the Jacobain and the wrench are defined in the same coordinate frame. In this thesis $W_{ext}$ is always defined in TCP frame and hence the Jacobain is also calculated in TCP frame. An algorithmic implementation is given below.

In [6]:
def convert_jacobian_to_tcp(jacobian_base, T_0_n):
    """
    Convert a Jacobian from the base frame to the TCP frame.
    
    :param jacobian_base: Jacobian matrix in the base frame (6 x 6)
    :param T_0_n: Transformation matrix from the base to the TCP (4 x 4)
    :return: Jacobian matrix (6 x 6) in the TCP frame
    """
    R_0_n = T_0_n[:3, :3]
    T_base_to_tcp = np.block([
        [R_0_n.T, np.zeros((3, 3))],
        [np.zeros((3, 3)), R_0_n.T]
    ])
    
    jacobian_tcp = np.dot(T_base_to_tcp, jacobian_base)
    return jacobian_tcp

## References

\begin{thebibliography}{4}
\bibitem{zotero|17740253/3J5DS9LQ}
Denavit, J., & Hartenberg, R. S. (2021). A Kinematic Notation for Lower-Pair Mechanisms Based on Matrices. {\em Journal of Applied Mechanics}, {\em 22}(2), 215–221. https://doi.org/10.1115/1.4011045


\bibitem{zotero|17740253/LY2FP5ZV}
Ghosal, A. (2006). {\em ROBOTICS: FUNDAMENTAL CONCEPTS AND ANALYSIS}. OUP India. https://books.google.de/books?id=GsRhAAAACAAJ

\end{thebibliography}